In [ ]:
import json
import numpy as np
from imageio import imread
import os
import random
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import colorsys

ORIGINAL_IMAGES_DIR = '/data/vein/original-images'
PREDICTIONS_JSON_PATH = '/data/dkpun-data/mask_rcnn/results/vein/predictions_20180621T114943/predictions.json'

def random_colors(N, bright=True):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def display_instances(image, instances, show_label=True):
    colors = random_colors(len(instances))
    
    _, ax = plt.subplots(figsize=(18, 18))
    ax.axis('off')
    
    for i, instance in enumerate(instances):
        color = colors[i]
        for verts in instance['polygons']:
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
            
        if show_label:
            x1, y1, x2, y2 = instance['bounding_box']
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                  alpha=0.7, linestyle="dashed",
                                  edgecolor=color, facecolor='none')
            ax.add_patch(p)

            caption = "vein {:.3f}".format(instance['score'])
            ax.text(x1, y1 + 8, caption, color='w', size=11, backgroundcolor="none")

    ax.imshow(image)
    plt.show()

predictions_json = json.load(open(PREDICTIONS_JSON_PATH))
for prediction in predictions_json:
    print("Filename: {}".format(prediction["original_image_id"]))
    image = imread(os.path.join(ORIGINAL_IMAGES_DIR, prediction["original_image_id"]))
    display_instances(image, prediction["vein_instances"], show_label=True)
    display_instances(image, prediction["vein_instances"], show_label=False)
